In [2]:
import sys
import os
sys.path.append(os.path.abspath('./voidFinderProject'))
from voidFinderProject import radius_analisis as ra

In [6]:
# =============================================================================
# IMPORTS
# =============================================================================
import pathlib
from voidfindertk import io, popcorn, svf_popcorn, core
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [ ]:
dataset_path = pathlib.Path("/home/jorgefederico/updates/vftk_1109/voidFinderProject/datasets")
woking_dir= pathlib.Path("/home/jorgefederico/updates/vftk_1109/voidFinderProject/run_popcorn")

d_09_dir = "tmpw990bohs2024-08-09T18:06:26.180577+00:00"
d_08_dir = "tmpovse6ej82024-08-21T16:12:49.132243+00:00"
d_07_dir = "tmplxhn4wcw2024-08-22T01:08:51.465883+00:00"
paths = [
    woking_dir / d_09_dir,
    woking_dir / d_08_dir,
    woking_dir / d_07_dir
]

In [ ]:
box = io.read_table(
    dataset_path/"halos_ascii_1000_1024_npmin_10_z0.51.dat",
    names=["m","x", "y", "z", "vx", "vy", "vz"])

In [ ]:
# pandas data from spherical popcorns
df9 = pd.read_csv(woking_dir / d_09_dir / "sphfile.dat",names=["idx","r","x","y","z","delta"], delim_whitespace=True)
df8 = pd.read_csv(woking_dir / d_08_dir / "sphfile.dat",names=["idx","r","x","y","z","delta"], delim_whitespace=True)
df7 = pd.read_csv(woking_dir / d_07_dir / "sphfile.dat",names=["idx","r","x","y","z","delta"], delim_whitespace=True)



In [9]:
path = pathlib.Path('/home/jorgefederico/updates/vftk_1109/voidFinderProject')
dens = [-0.9,-0.8,-0.7]
models = [
    path / f"svf_popcorn_model_2{dens[0]}.pkl",
    path / f"svf_popcorn_model_2{dens[1]}.pkl",
    path / f"svf_popcorn_model_2{dens[2]}.pkl",
]

In [10]:
model09 = joblib.load(models[0])
model08 = joblib.load(models[1])
model07 = joblib.load(models[2])


In [13]:
ef_rad_08 = model08.effective_radius()
ef_rad_07 = model07.effective_radius()
ef_rad_09 = model09.effective_radius()


In [ ]:
# Analisis 09
# indice errores
e = np.where(ef_rad_09.errors >0)[0]
## indice no errores
ne = np.where(ef_rad_09.errors ==0)[0]

# radios validos con discrepancias
# radios popcorn
r = np.array(df9["r"])[ne] #df9 dataframe con los datos de svf popcorn
# radios calculados por effective_rad
r9 = ef_rad_09.radius[ne]

# indices de radios discrepantes
disc = np.where(np.abs(r-r9)>3)[0]

# contraste de densidad integrada
delta = (1+dens[0])*(len(box)/(box.size()**3)) #NO SE TOCA

In [ ]:
#disc index
disc_indx = 0
# exploro mapa de densidad
dens_ = ef_rad_09.densities[ne]


dens_map = dens_[disc[disc_indx]]
n = np.arange(1,len(dens_map)+1)

# map the popcorn rad to n
n_map = n[np.where(np.abs(n-dens_map*((4/3)*np.pi*(r[disc[disc_indx]]**3)))<0.9)[0]]
# map the my value rad to n
n_map2 = n[np.where(np.abs(n-dens_map*((4/3)*np.pi*(r9[disc[disc_indx]]**3)))<0.0195)[0]]
#plot
dots = 40

# get radius from density map
rad = (n/((4/3)*np.pi*dens_map))**(1/3)

In [ ]:
fix,ax = plt.subplots(1,1,figsize=(12,5))

########################
ax.scatter(rad[:dots],dens_map[:dots]/(len(box)/(box.size()**3)), s=10.5)
ax.axhline(y=delta/(len(box)/(box.size()**3)),color="r")
ax.axvline(x=r[disc[disc_indx]],c="g")
ax.axvline(x=r9[disc[disc_indx]],c="b")
ax.text(r[disc[disc_indx]],delta+0.2*delta,f"SVFPopCorn \n r={r[disc[disc_indx]]:.2f}")
ax.text(r9[disc[disc_indx]],delta-0.5*delta,f"R_eff Method \n r={r9[disc[disc_indx]]:.2f}")
ax.set_xlabel(r"$radius$")
#########################
plt.savefig("analisis_plot.jpg")
plt.close()

Basicamente la idea es que en estos 44 ejemplos donde yo no tendria un radio, popcorn detecta que hay un radio. Segun recuerdo popcorn no elije radios a menos que deje por dentro algunas particulas. Asi que la hipotesis es que aunque agregue la periodicidad no estaria viendo algunas particulas.

$\delta = \frac{\rho}{\rho_{med}} - 1$
$$(\delta+1)\rho_{med}=\rho$$

- El valor de rho medio lo puedo calcular con la caja: $\rho_{med}=\frac{N}{l_{x}l_{y}l_{z}}$


In [ ]:
# De los centros que encontre dame sus indices asi puedo buscar sus posiciones x,y,z.
# e me da el indice de los errores. Ahora busco sus coordenadas con el dataframe:
df_e = df9.loc[df9.index.intersection(e)]

# La idea es que uno puede estimar el valor de n considerando los valores de densidad usando la ecuacion anterior:
df_e["density"] = ((np.array(df_e['delta']))+1)*(len(box)/(box.size()**3))

Ahora si puedo calcular la cantidad de particulas que estarian adentro:
$\rho = \frac{N}{\frac{4\pi r^{3}}{3}}$
$$\frac{\rho 4 \pi r^{3}}{3}=N$$

In [ ]:
# Hacemos entonces el calculo de arriba
n_part = np.round(df_e['density']*(4*np.pi/3)*(df_e['r']**3),2)
# Con esto llegaria a la conclusion de que no deberian haber particulas en estos voids.
df_e["n_tracers"] = n_part
df_e.reset_index(inplace=True)

Sin embargo tengo que ser consistente con lo que estoy calculando, asi que priver voy a tener que verificar que lo que estoy haciendo es correcto cuando considero elementos validos.

In [ ]:
# Veamos los no errores entonces:
df_ne = df9.loc[df9.index.intersection(ne)]
# Calculemos las densidades:
df_ne["density"] = ((np.array(df_ne['delta']))+1)*(len(box)/(box.size()**3))
# Finalmente calculamos el numero de particulas:
n_part_ne = np.round(df_ne['density']*(4*np.pi/3)*(df_ne['r']**3),2)

In [ ]:
eindx = 5
# exploro mapa de densidad
dens_ = ef_rad_09.densities[e]
dens_map = np.round(dens_[eindx],6)
rad = (n/((4/3)*np.pi*dens_map))**(1/3)

####
dots = 40
fig,ax = plt.subplots(1,1)
ax.scatter(rad[:dots],dens_map[:dots]/(len(box)/(box.size()**3)),s=5.5)
ax.axhline(y=delta/(len(box)/(box.size()**3)),color="r")
ax.set_ylabel(r"$\rho$")
ax.set_xlabel(r"$R$")
plt.savefig("errors_plot.jpg")
plt.close()

In [ ]:


# compare radius
(r[disc[disc_indx]],r9[disc[disc_indx]])


In [ ]:
#check distances
x = box.arr_.x
y = box.arr_.y
z = box.arr_.z
xyz = np.column_stack((x,y,z))

#centers
xyz_c = np.array(df9[["x","y","z"]])[ne][disc[disc_indx]]

d = np.sort(cdist([xyz_c],xyz)[0])

#recalculo todo
n_neighbors= 100
n = np.arange(1, n_neighbors + 1)
den = (3 * n) / (4 * np.pi * d[:n_neighbors]**3)
fig,ax = plt.subplots(1,1)
ax.scatter(n[:dots],den[:dots], s=9.5)
ax.axhline(y=delta,color="r")
plt.savefig("dencheck.jpg")
plt.close()

# Analisis con los otros dataset
Vamos a ver si no existe algo extranio en los otros dataset.

In [ ]:
def get_effectiv_rad(path,dens,box):
    """Genera un modelo de popcorn esferico y lo guarda en un archivo .pkl
    Parameters
    ----------
        path: Path hacia el directiorio donde se encuentran los archivos de la
        corrida de SVF popcorn. Comunmente denominado run_work_dir.
        dens: Valor de la delta: -0.9,-0.8,-0.7 son valores tipicos.
        box: Caja con los trazadores
    Returns
    -------
        None.
    """

    model = svf_popcorn.SVFPopCorn(
        workdir=path,
        boxsize=1000,
        densth=dens,
        minradius = 5,
        maxradius = 100
    )
    build_voids_parameters = {"run_work_dir": path, "box": box}
    tracers, centrs, extra = model.build_voids(build_voids_parameters)

    voids = core.voids.Voids(
    method="Popcorn",
    box=box,
    tracers_in_voids_=tracers,
    centers_=centrs,
    extra_=extra,
    )
    joblib.dump(voids, f"svf_popcorn_model_2{dens}.pkl")

In [ ]:
import contextlib
import time
@contextlib.contextmanager
def take_time():
    start_time = time.time()
    yield start_time
    print("elapsed: ",time.time()-start_time)

In [ ]:
#Genero un nuevo modelo para delta = -0.8
# Recordatorio
# paths = [
#     woking_dir / d_09_dir,
#     woking_dir / d_08_dir,
#     woking_dir / d_07_dir
# ]
get_effectiv_rad(paths[1],dens=-0.8,box=box)

# Cargo el modelo que genere:
model08 = joblib.load(models[1])

# Levanto su dataframe correspondiente:
df8 = pd.read_csv(woking_dir / d_08_dir / "sphfile.dat",names=["idx","r","x","y","z","delta"], delim_whitespace=True)

# Obtenemos los radios efectivos correspondientes
with take_time() as tt:
    ef_rad_08 = model08.effective_radius(delta=-0.8,n_neighbors=100)
# Tener en cuenta que mas o menos erroes pueden aparecer dependiendo del numero de vecinos hechos en tu busqueda.
# Tambien puede influir la delta que estas utilizando
# indice errores
e = np.where(ef_rad_08.errors >0)[0]
# indice no errores
ne = np.where(ef_rad_08.errors ==0)[0]

In [ ]:
# radios validos con discrepancias

# radios popcorn
r = np.array(df8["r"])[ne] #df9 dataframe con los datos de svf popcorn
# radios calculados por effective_rad
r8 = ef_rad_08.radius[ne]

# indices de radios discrepantes
disc = np.where(np.abs(r-r8)>3)[0]


In [ ]:
#disc index
disc_indx = 3
# exploro mapa de densidad
dens_ = ef_rad_08.densities[ne]


dens_map = dens_[disc[disc_indx]]
n = np.arange(1,len(dens_map)+1)

# get radius from density map
rad = (n/((4/3)*np.pi*dens_map))**(1/3)
delta = (1+dens[1])*(len(box)/(box.size()**3)) #NO SE TOCA

In [ ]:
fix,ax = plt.subplots(1,1,figsize=(12,5))
dots=70
########################
ax.scatter(rad[:dots],dens_map[:dots]/(len(box)/(box.size()**3)), s=10.5)
ax.axhline(y=delta/(len(box)/(box.size()**3)),color="r")
ax.axvline(x=r[disc[disc_indx]],c="g")
ax.axvline(x=r8[disc[disc_indx]],c="b")
ax.text(r[disc[disc_indx]],0.3,f"SVFPopCorn \n r={r[disc[disc_indx]]:.2f}")
ax.text(r8[disc[disc_indx]],0.2,f"R_eff Method \n r={r8[disc[disc_indx]]:.2f}")
ax.set_xlabel(r"$radius$")
#########################
plt.savefig("analisis_plot.jpg")
plt.close()

In [ ]:
eindx = 2
# exploro mapa de densidad
dens_ = ef_rad_08.densities[e]
dens_map = np.round(dens_[eindx],6)
rad = (n/((4/3)*np.pi*dens_map))**(1/3)

####
dots = 70
fig,ax = plt.subplots(1,1)
ax.scatter(rad[:dots],dens_map[:dots]/(len(box)/(box.size()**3)),s=5.5)
ax.axhline(y=delta/(len(box)/(box.size()**3)),color="r")
ax.axvline(x=df8['r'][e[eindx]])
ax.text(df8['r'][e[eindx]],0.2,f"PopCorn \n r={df8['r'][e[eindx]]:.2f}")
ax.set_ylabel(r"$\rho$")
ax.set_xlabel(r"$R$")
plt.savefig("errors_plot.jpg")
plt.close()

Como veo cosas raras tengo que acudir al dataframe y asegurarme que veo lo mismo

In [ ]:
df_e = df8.loc[df8.index.intersection(e)]

# La idea es que uno puede estimar el valor de n considerando los valores de densidad usando la ecuacion anterior:
df_e["density"] = ((np.array(df_e['delta']))+1)*(len(box)/(box.size()**3))

# errores
n_part = np.round(df_e['density']*(4*np.pi/3)*(df_e['r']**3),2)
df_e["n_tracers"] = n_part
df_e.reset_index(inplace=True)
df_e.loc[eindx,:]

# Veamos los no errores entonces:
df_ne = df8.loc[df8.index.intersection(ne)]
# Calculemos las densidades:
df_ne["density"] = ((np.array(df_ne['delta']))+1)*(len(box)/(box.size()**3))
# Finalmente calculamos el numero de particulas:
n_part_ne = np.round(df_ne['density']*(4*np.pi/3)*(df_ne['r']**3),2)
df_ne["n_tracers"] = n_part_ne
df_ne.reset_index(inplace=True)

# Busco en el dataframe segun las discrepancias
df_ne.loc[disc[disc_indx],:]


Debugging

In [ ]:
from voidfindertk.core import vsf
centers_xyz = np.array(df8[['x','y','z']])
vsf.effective_radius(centers_xyz, box, delta=-0.9, n_neighbors=100, n_cells=64)
